# United States Emissions Dashboard (EPA 2023)

This notebook contains SQL queries and visualizations for analyzing US greenhouse gas emissions.  
Data source: EPA 2023 emissions dataset.

---

## Setup

In [0]:
%sql
-- Set database context
USE emissions.default;

## Geospatial Emissions Mapping
Visualize emissions geographically across the continental US.

In [0]:
%sql
-- Query 1: Geospatial Emissions Mapping
SELECT latitude AS lat,
       longitude AS lon,
       `GHG emissions mtons CO2e` AS Emissions
FROM emissions_data
ORDER BY Emissions
LIMIT 500;

Databricks visualization. Run in Databricks to view.

## Emissions Per Person by County
Normalize emissions by population to highlight efficiency or intensity.

In [0]:
%sql
-- Query 2: Emissions Per Person by County
SELECT county_state_name,
       population,
       CAST(REPLACE(`GHG emissions mtons CO2e`, ',', '') AS DOUBLE) /
       CAST(REPLACE(population, ',', '') AS DOUBLE) AS Emissions_per_person
FROM emissions_data
ORDER BY Emissions_per_person DESC;

Databricks visualization. Run in Databricks to view.

## Top 10 States by Total Emissions
Aggregate emissions at the state level.

In [0]:
%sql
-- Query 3: Top 10 States by Total Emissions
SELECT state_abbr,
       SUM(CAST(REPLACE(`GHG emissions mtons CO2e`, ',', '') AS DOUBLE)) AS Total_Emissions
FROM emissions_data
GROUP BY state_abbr
ORDER BY Total_Emissions DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

## Top 10 Counties by Total Emissions
Highlight counties with the highest emissions.

In [0]:
%sql
-- Query 4: Top 10 Counties by Total Emissions
SELECT county_state_name,
       SUM(CAST(REPLACE(`GHG emissions mtons CO2e`, ',', '') AS DOUBLE)) AS Total_Emissions
FROM emissions_data
GROUP BY county_state_name
ORDER BY Total_Emissions DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

## Aggregate Contribution of Top 10 States
Calculate the share of total US emissions from the top 10 states.

In [0]:
%sql
-- Query 5: Aggregate Contribution of Top 10 States
WITH state_totals AS (
    SELECT state_abbr,
           SUM(CAST(REPLACE(`GHG emissions mtons CO2e`, ',', '') AS DOUBLE)) AS total_emissions
    FROM emissions_data
    GROUP BY state_abbr
),
us_total AS (
    SELECT SUM(CAST(REPLACE(`GHG emissions mtons CO2e`, ',', '') AS DOUBLE)) AS us_emissions
    FROM emissions_data
),
top_10 AS (
    SELECT st.state_abbr,
           st.total_emissions,
           (st.total_emissions / ut.us_emissions) * 100 AS pct_of_us
    FROM state_totals st
    CROSS JOIN us_total ut
    ORDER BY st.total_emissions DESC
    LIMIT 10
),
agg AS (
    SELECT SUM(total_emissions) AS total_emissions_top_10,
           SUM(pct_of_us) AS pct_top_10
    FROM top_10
)
SELECT total_emissions_top_10, pct_top_10
FROM agg;

Databricks visualization. Run in Databricks to view.

## Key Insights
- Top 10 states contribute ~51% of total US emissions.
- Higher population areas tend to have lower emissions per person.
- Emissions are concentrated in specific counties (Maricopa, Harris, Cook).
- Geographic mapping reveals dense emission clusters around major metropolitan areas.